In [1]:
#installing dependencies
!pip install -q transformers datasets peft bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 31.0 MB/s eta 0:00:00:00:0100:01


In [2]:
#imports
import warnings
warnings.filterwarnings("ignore")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from peft import PeftModel


2026-01-30 05:33:11.522121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769751191.961476      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769751192.099316      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769751193.200763      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769751193.200807      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769751193.200810      55 computation_placer.cc:177] computation placer alr

In [10]:
#Loading the model 
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
#lora-config for fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [16]:
#importing the datasets for finetuning
import pandas as pd
df = pd.read_csv("/kaggle/input/imdb-movies-dataset/imdb_movies.csv")
df.head()


data = []

for _, row in df.iterrows():
    title = row["names"]
    overview = row["overview"]
    genre = row["genre"]
    score = row["score"]

    # Q1: plot explanation
    q1 = f"What is the movie {title} about?"
    a1 = overview

    data.append({
        "text": f"### Question: {q1}\n### Answer: {a1}"
    })

    # Q2: genre-based recommendation
    q2 = f"Recommend a {genre} movie."
    a2 = f"{title} is a good recommendation. It has a user score of {score}. {overview}"

    data.append({
        "text": f"### Question: {q2}\n### Answer: {a2}"
    })



,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


In [ ]:
#converting datasets into HuggingFace datasets 
from datasets import Dataset

# convert list to dataset first if not already
full_dataset = Dataset.from_list(data)

# shuffle + take subset
dataset = full_dataset.shuffle(seed=42).select(range(6000))

len(dataset)


In [ ]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

dataset = dataset.map(tokenize, batched=False)
dataset = dataset.remove_columns(["text"])
dataset.set_format("torch")


In [ ]:
#Training args 
training_args = TrainingArguments(
    output_dir="./movie-llm",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,

    logging_strategy="steps",   # 👈 force logging
    logging_steps=1,            # 👈 every step
    logging_first_step=True,

    disable_tqdm=True,          # 👈 KILLS progress bar (important)
    report_to="none",           # 👈 no wandb / tensorboard
    save_strategy="no",
    log_level="info"
)


In [ ]:
#Fine tuning the model 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)
print("🚀 TRAINING STARTED")
trainer.train()
print("✅ TRAINING FINISHED")


In [ ]:
#saving the finetuned model 
trainer.model.save_pretrained("movie-lora-adapter")
tokenizer.save_pretrained("movie-lora-adapter")


After saving the finetuned model we are loading that model as LORA_PATH and testing it.

In [9]:
!ls /kaggle/input/d/jeevaaaa/movie-lora/movie-lora-adapter
LORA_PATH = "/kaggle/input/d/jeevaaaa/movie-lora/movie-lora-adapter"


adapter_config.json	   README.md		    tokenizer.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.model
chat_template.jinja	   tokenizer_config.json


In [8]:
!unzip /kaggle/input/movie-lora-adapter/movie-lora-adapter.zip


unzip:  cannot find or open /kaggle/input/movie-lora/movie-lora-adapter.zip, /kaggle/input/movie-lora/movie-lora-adapter.zip.zip or /kaggle/input/movie-lora/movie-lora-adapter.zip.ZIP.


In [4]:
!ls movie-lora-adapter


ls: cannot access 'movie-lora-adapter': No such file or directory


In [13]:
#using the finetuned model 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

model = PeftModel.from_pretrained(
    model,
    LORA_PATH
)

model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(32000, 2048)
            (layers): ModuleList(
              (0-21): 22 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [17]:
#testing the finetuned model 
prompt = """### Question: Recommend a science fiction movie with a twist.
### Answer:"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=True,          # 👈 IMPORTANT
        temperature=0.4,         # low = controlled
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Question: Recommend a science fiction movie with a twist.
### Answer: "The Martian" (2015) - A film about an astronaut stranded on Mars who must use his ingenuity and resourcefulness to survive. The twist is that the astronaut's wife was killed in a tragic accident before he left Earth, and he had to make his way back to Earth alone. This movie is a great example of how a twist can add depth and complexity to a story.
